# User-Based Collaborative Filtering for Book Recommendation

In [2]:
import numpy as np
import pandas as pd

In [86]:
df_books_final = pd.read_csv('df_books_final.csv')
df_train_interactions = pd.read_csv('train_interactions.csv')
df_test_interactions = pd.read_csv('test_interactions.csv')

In [22]:
df_train_interactions = pd.read_csv('train_interactions.csv')

In [27]:
df_train_interactions.head(3)

,user_id,book_id,rating,n_votes,review_age,sentiment
0,1,39343,2,0.0,3434,0.851100
1,1,21648,0,NaN,3435,0.406046
2,2,7794,4,0.0,3046,-0.421500


In [28]:
df_train_interactions = pd.read_csv('train_interactions2.csv')

In [29]:
df_train_interactions.head(3)

,user_id,book_id,rating,n_votes,review_age,sentiment
0,0,2497,0,NaN,2802,0.406046
1,0,5556,4,0.0,2809,-0.224400
2,0,1330,4,0.0,2888,0.808300


In [31]:
df_train_interactions.describe()

,user_id,book_id,rating,n_votes,review_age,sentiment
count,885101.000000,885101.000000,885101.000000,149268.000000,885101.000000,885101.000000
mean,16861.607208,31296.160164,3.760184,0.724288,3872.041447,0.482490
std,9930.411638,30903.590974,1.236461,4.371020,891.109311,0.216447
min,0.000000,0.000000,0.000000,-2.000000,2531.000000,-0.989000
25%,8084.000000,4348.000000,3.000000,0.000000,3142.000000,0.406860
50%,16738.000000,17416.000000,4.000000,0.000000,3768.000000,0.527973
75%,25436.000000,59604.000000,5.000000,0.000000,4451.000000,0.574139
max,34192.000000,97368.000000,5.000000,493.000000,6962.000000,0.999700


In [3]:
df_books_final.head(3)

,book_id,title,description,average_rating,ratings_count,text_reviews_count,top_popular_shelves,author_ids,format_Audio,format_Digital,...,lang_tha,lang_tr,lang_tur,lang_ukr,lang_vi,lang_vie,lang_zh,length_long,length_medium,length_short
0,1882090,"Behave Yourself, Bethany Brant",A preacher's daughter with lots of curiosity a...,-1.028630,-0.059069,-0.136278,"[{'count': '1', 'name': 'mrs-withdrawn'}, {'co...",['151369'],False,False,...,False,False,False,False,False,False,False,False,True,False
1,166120,Ghosthunters And The Incredibly Revolting Ghost,A $2.99 value-priced edition of one of our bes...,-0.690046,-0.059069,-0.126186,"[{'count': '9', 'name': 'owned'}, {'count': '9...",['15873'],False,False,...,False,False,False,False,False,False,False,False,True,False
2,8608741,Attack of the Chicken Nugget Man: A National T...,Third-grader Chris Robb just can't seem to do ...,-0.597705,-0.059069,-0.102639,"[{'count': '2', 'name': 'books-i-wrote'}, {'co...",['2868520'],False,False,...,False,False,False,False,False,False,False,False,True,False


In [68]:
df_train_interactions.head(5)

user_id  book_id  rating  n_votes  review_age  sentiment
0        0     2497       0      NaN        2802   0.406046
1        0     5556       4      0.0        2809  -0.224400
2        0     1330       4      0.0        2888   0.808300
3        0      177       4      0.0        2907   0.874500
4        1    92062       4      NaN        2549   0.527973

In [34]:
user_item_interactions = df_train_interactions[['user_id', 'book_id', 'rating']]
user_item_interactions

,user_id,book_id,rating
0,0,2497,0
1,0,5556,4
2,0,1330,4
3,0,177,4
4,1,92062,4
...,...,...,...
885096,34192,72240,5
885097,34192,34391,4
885098,34192,782,3
885099,34192,84,4


In [4]:
# number of neighbors of user
N_NEIGHBORS = 10

# number of recommendations
N_RECOMMENDATIONS = 5

In [87]:
def read_ratings(df):
    """
    Read the raw data of the book ratings.

    Returns a list of tuples:
    (user id, book_id, rating)
    """

    data = []
    for _, row in df.iterrows():
        user_id = row['user_id']
        book_id = row['book_id']
        rating = row['rating']
        data.append((user_id, book_id, rating))

    return data

In [88]:
ratings = read_ratings(df_train_interactions)
ratings = pd.DataFrame(data=ratings, columns=['user', 'book', 'rating'])
ratings.head()

,user,book,rating
0,0,2218,5
1,0,3922,4
2,0,762,4
3,0,18839,5
4,0,2395,4


In [36]:
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder

In [37]:
encoder = LabelEncoder()
user_item_interactions['user_idx'] = encoder.fit_transform(user_item_interactions['user_id'])
user_item_interactions['item_idx'] = encoder.fit_transform(user_item_interactions['book_id'])

n_users = len(np.unique(user_item_interactions['user_idx']))
n_items = len(np.unique(user_item_interactions['item_idx']))

# Create sparse matrix (rows: users, columns: items)
affinity_matrix = csr_matrix((user_item_interactions['rating'],
                             (user_item_interactions['user_idx'], user_item_interactions['item_idx'])),
                             shape=(n_users, n_items))

C:\Users\rayna\AppData\Local\Temp\ipykernel_46824\2387205276.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_item_interactions['user_idx'] = encoder.fit_transform(user_item_interactions['user_id'])
C:\Users\rayna\AppData\Local\Temp\ipykernel_46824\2387205276.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_item_interactions['item_idx'] = encoder.fit_transform(user_item_interactions['book_id'])


In [18]:
# number of unique books
n_books = ratings.book.nunique()
print('Number of Unique Books:', n_books)

Number of Unique Books: 47682


In [20]:
# number of unique users
n_users = ratings.user.nunique()
print('Number of Unique Users:', n_users)

Number of Unique Users: 15398


### Transforming the user rating data to wide format

In [11]:
ratings.head(3)

,user,book,rating
0,0.0,2497.0,0.0
1,0.0,5556.0,4.0
2,0.0,1330.0,4.0


In [18]:
ratings[ratings < 0].notna().sum()

user      0
book      0
rating    0
dtype: int64

In [89]:
# ratings_raw = ratings.copy()
ratings = ratings.pivot(index='user', columns='book', values='rating')

In [33]:
duplicates = df_train_interactions[df_train_interactions.duplicated(subset=['user_id', 'book_id'], keep=False)]
print(duplicates)

Empty DataFrame
Columns: [user_id, book_id, rating, n_votes, review_age, sentiment]
Index: []


In [ ]:
ratings = df_train_interactions.pivot(index='user_id', columns='book_id', values='rating')

In [13]:
ratings.shape

(885101, 3)

In [26]:
ratings.head(10)

book,5,50,93,236,244,302,314,330,881,903,...,35297101,35429280,35440603,35479935,35546694,35616438,35757340,35757419,36131198,36311957
user,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Creating Affinity Matrix Based on Ratings

In [22]:
from scipy.sparse import csr_matrix

In [27]:
user_book_matrix = ratings.fillna(0)

In [28]:
user_book_matrix.head(10)

book,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,57048.0,57049.0,57050.0,57051.0,57052.0,57053.0,57054.0,57055.0,57056.0,57057.0
user,,,,,,,,,,,,,,,,,,,,,
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
affinity_matrix = csr_matrix(user_book_matrix.values)

### Similarity between users

In [39]:
def pearson_similarity(v1, v2):
    """
    Compute the Pearson correlation between to ratings vectors.

    pd.corr() function can handle missing data.

    parameters:
    - v1, v2: pd.Series, ratings vectors

    returns:
    - float

    """

    pearson = v1.corr(v2)
    # pearson = user_book_matrix.T.corr(method='pearson')

    return pearson

In [40]:
def compute_similarities(user_id, ratings_matrix):
    """
    Compute the similarity of a given user with all the other users in the dataset.

    Remove the similarity value for the given user from the result.

    returns:
        - a pd.Series with the user id's as index, and similarity as series values
    """

    # get ratings of user to re-use in the similarity computation
    ratings_user = ratings_matrix.loc[user_id,:]

    # calculate the similarity between the given user and the other users
    similarities = ratings_matrix.apply(
        lambda row: pearson_similarity(ratings_user, row),
        axis=1)

    similarities = similarities.to_frame(name='similarity')

    # find most similar users to the given user
    similarities = similarities.sort_values(by='similarity', ascending=False)

    # drop the similarity of the user (should be ~1 anyways)
    similarities = similarities.drop(user_id)

    return similarities

In [60]:
def compute_similarities(ratings_matrix):
    """
    Compute the similarity matrix between all users in the dataset using a CSR matrix.
    
    Args:
        ratings_matrix: The ratings matrix in CSR format (sparse matrix).
    
    Returns:
        A pandas DataFrame with users as both row and column indices, and similarity as values.
    """
    # Get the number of users
    num_users = ratings_matrix.shape[0]
    
    # Initialize an empty similarity matrix (size: num_users x num_users)
    similarity_matrix = np.zeros((num_users, num_users))
    
    # Iterate through each pair of users
    for user_id in range(num_users):
        # Extract the ratings of the given user (row in the CSR matrix)
        user_ratings = ratings_matrix[user_id].toarray().flatten()

        for other_user_id in range(user_id + 1, num_users):  # To avoid redundant calculations
            # Extract the ratings of the other user (row in the CSR matrix)
            other_user_ratings = ratings_matrix[other_user_id].toarray().flatten()

            # Compute the Pearson similarity between the two users
            similarity = pearson_similarity(user_ratings, other_user_ratings)

            # Assign the computed similarity to both [user_id, other_user_id] and [other_user_id, user_id]
            similarity_matrix[user_id, other_user_id] = similarity
            similarity_matrix[other_user_id, user_id] = similarity

    # Convert the similarity matrix to a pandas DataFrame
    similarity_df = pd.DataFrame(similarity_matrix, index=range(num_users), columns=range(num_users))
    
    return similarity_df

def pearson_similarity(user_ratings, other_user_ratings):
    """
    Compute Pearson similarity between two rating vectors.
    
    Args:
        user_ratings: A 1D numpy array of ratings for the given user.
        other_user_ratings: A 1D numpy array of ratings for the other user.
        
    Returns:
        A Pearson similarity value between -1 and 1.
    """
    # Only consider the ratings that are not zero (i.e., non-missing ratings)
    mask = (user_ratings > 0) & (other_user_ratings > 0)
    
    # Extract the ratings that are common (both users rated the same item)
    common_ratings_user = user_ratings[mask]
    common_ratings_other_user = other_user_ratings[mask]
    
    if len(common_ratings_user) > 1:
        # Calculate Pearson correlation
        return np.corrcoef(common_ratings_user, common_ratings_other_user)[0, 1]
    else:
        # If not enough common ratings, return NaN or 0
        return np.nan


In [63]:
user_item_interactions

,user_id,book_id,rating,user_idx,item_idx
0,0,2497,0,0,2358
1,0,5556,4,0,5222
2,0,1330,4,0,1270
3,0,177,4,0,169
4,1,92062,4,1,82304
...,...,...,...,...,...
885096,34192,72240,5,34192,64111
885097,34192,34391,4,34192,31343
885098,34192,782,3,34192,753
885099,34192,84,4,34192,79


In [81]:
vec = affinity_matrix[0].toarray().flatten()

In [82]:
vec[vec != 0]

array([4, 4, 4], dtype=int64)

In [61]:
user_similarity_matrix = compute_similarities(affinity_matrix)

C:\Users\rayna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\rayna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


KeyboardInterrupt: 

In [180]:
user_similarity_matrix

pandas.core.frame.DataFrame

In [239]:
book_ratings = ratings[39343].dropna()
user_ids = book_ratings.index
user_vectors = ratings.loc[user_ids]

In [240]:
user_ids

Index([1.0, 9747.0], dtype='float64', name='user')

In [244]:
user_vectors

book,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,57048.0,57049.0,57050.0,57051.0,57052.0,57053.0,57054.0,57055.0,57056.0,57057.0
user,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9747.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [260]:
user1 = user_vectors.iloc[0]
user9747 = user_vectors.iloc[1]

In [267]:
user1.corr(user9747)

C:\Users\rayna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\rayna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
C:\Users\rayna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


nan

In [268]:
# Select the two user vectors
user1_vector = user_vectors.iloc[0]
user9747_vector = user_vectors.iloc[1]

# Drop NaN values from both user vectors to get only the common ratings
common_ratings = user1_vector.dropna().index.intersection(user9747_vector.dropna().index)

# Check if there are at least two overlapping ratings
if len(common_ratings) >= 2:
    # Calculate correlation using only the common ratings
    similarity = user1_vector[common_ratings].corr(user9747_vector[common_ratings])
    print("Pearson correlation between User 1 and User 9747:", similarity)
else:
    print("Not enough overlapping ratings to calculate Pearson correlation.")


Not enough overlapping ratings to calculate Pearson correlation.


In [231]:
ratings[39343].notna()

user
1.0         True
2.0        False
3.0        False
4.0        False
5.0        False
           ...  
16894.0    False
16895.0    False
16896.0    False
16897.0    False
16898.0    False
Name: 39343.0, Length: 15398, dtype: bool

In [238]:
sum(ratings[39343].notna())

2

In [235]:
ratings[ratings[39343].notna()]

book,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,57048.0,57049.0,57050.0,57051.0,57052.0,57053.0,57054.0,57055.0,57056.0,57057.0
user,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9747.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [229]:
ratings.iloc[9747][39343]

nan

In [193]:
user_sim = ratings.iloc[0]
user_sim[user_sim.notna()]
# user_sim

book
21648.0    0.0
39343.0    2.0
Name: 1.0, dtype: float64

In [44]:
sim_scores

31723    1.0
11954    1.0
5075     1.0
20881    1.0
14154    1.0
        ... 
34188    NaN
34189    NaN
34190    NaN
34191    NaN
34192    NaN
Length: 34192, dtype: float64

In [47]:
sim_scores = compute_similarities(2, affinity_matrix)
sim_values = sim_scores[sim_scores.notna()]

C:\Users\rayna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\rayna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [48]:
sim_values.describe()

count    4904.000000
mean        0.128089
std         0.698966
min        -1.000000
25%        -0.500000
50%         0.238271
75%         0.790569
max         1.000000
dtype: float64

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
user_similarity_matrix = cosine_similarity(affinity_matrix, dense_output=True)

NameError: name 'cosine_similarity' is not defined

In [161]:
userid = 11
sim_scores = pd.DataFrame(user_similarity_matrix)[userid]
nonzero_idx = np.nonzero(pd.DataFrame(user_similarity_matrix)[userid])[0]
nonzero_idx = nonzero_idx[nonzero_idx != userid]
sim_scores[sim_scores.index.isin(nonzero_idx)].describe()
# sim_scores
# nonzero_idx

count    1372.000000
mean        0.071419
std         0.066693
min         0.005656
25%         0.037759
50%         0.054378
75%         0.081319
max         0.779266
Name: 11, dtype: float64

In [136]:
nonzero_idx = np.nonzero(pd.DataFrame(user_similarity_matrix)[2])[0]

In [140]:
print(len(nonzero_idx))
pd.DataFrame(user_similarity_matrix[nonzero_idx])

604


,0,1,2,3,4,5,6,7,8,9,...,15388,15389,15390,15391,15392,15393,15394,15395,15396,15397
0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.104494,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.078723
2,0.0,0.0,0.062871,0.039088,0.0,0.0,0.0,0.043532,0.000000,0.0,...,0.000000,0.0,0.077943,0.0,0.0,0.098085,0.099427,0.000000,0.000000,0.000000
3,0.0,0.0,0.039130,0.032437,0.0,0.0,0.0,0.117043,0.085504,0.0,...,0.089658,0.0,0.045276,0.0,0.0,0.119380,0.049505,0.000000,0.089658,0.029479
4,0.0,0.0,0.218218,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.164399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,0.0,0.0,0.070911,0.047026,0.0,0.0,0.0,0.091650,0.000000,0.0,...,0.000000,0.0,0.179016,0.0,0.0,0.129804,0.000000,0.011472,0.129983,0.032053
600,0.0,0.0,0.023661,0.019614,0.0,0.0,0.0,0.029125,0.038298,0.0,...,0.000000,0.0,0.007111,0.0,0.0,0.017500,0.000000,0.000000,0.072287,0.053477
601,0.0,0.0,0.071518,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
602,0.0,0.0,0.114123,0.000000,0.0,0.0,0.0,0.000000,0.173175,0.0,...,0.000000,0.0,0.075027,0.0,0.0,0.063304,0.000000,0.000000,0.261488,0.193448


In [96]:
user_similarity_matrix.shape

(34193, 34193)

In [98]:
pd.DataFrame(user_similarity_matrix)

,0,1,2,3,4,5,6,7,8,9,...,34183,34184,34185,34186,34187,34188,34189,34190,34191,34192
0,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.113228,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.379663
1,0.000000,1.000000,0.025319,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.042919,0.032940,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
2,0.000000,0.025319,1.000000,0.210824,0.0,0.157115,0.035049,0.027522,0.0,0.000000,...,0.0,0.034774,0.049108,0.109562,0.0,0.122592,0.0,0.000000,0.036577,0.034606
3,0.000000,0.000000,0.210824,1.000000,0.0,0.076732,0.000000,0.000000,0.0,0.097346,...,0.0,0.000000,0.110531,0.000000,0.0,0.105773,0.0,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34188,0.000000,0.000000,0.122592,0.105773,0.0,0.066432,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.000000,0.000000,0.000000
34189,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.000000
34190,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.150424
34191,0.000000,0.000000,0.036577,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.047587,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000


### Predict Book Ratings

In [83]:
def predict_rating(item_id, ratings, similarities, N=10):
    """
    Predict the rating of a given item by a user, given the ratings of similar users.
    Takes the N users with the highest similarity measure, AND who have rated the given item.
    Returns the average rating of the most similar users who previously rated the item.

    parameters:
    - item_id: int, item that needs a rating prediction
    - ratings: pd.DataFrame
    - similarities: pd.DataFrame
    - N: int, number of neighbors to use for rating prediction

    returns:
    - a float representing the predicted rating for the given item

    """

    # get the ratings of all users for the specific item
    users_ratings = ratings.loc[:, item_id]

    # only keep users who rated the given item, otherwise you won't be able to generate a prediction based on the users ratings
    most_similar_users_who_rated_item = similarities.loc[~users_ratings.isnull()]

    # keep N users with highest similarities to given user who also rated the given item
    N_most_similar_users = most_similar_users_who_rated_item.head(N)

    # find ratings item for most similar users:
    ratings_for_item = ratings.loc[N_most_similar_users.index, item_id]

    # predict the rating of the item by averaging the ratings of that item of the most similar users
    return ratings_for_item.mean(), len(ratings_for_item)

In [36]:
user_item_interactions.head(3)

,user_id,book_id,rating
0,1,39343,2
1,1,21648,0
2,2,7794,4


#### Parameters that can be set

In [129]:
k = 10 # top k similar users
sim_threshold = 0.1

In [57]:
def get_collaborative_filtering_recommendations(user_similarity_matrix, customer_id, k=10, sim_threshold = 0.6):

    # Access the globally defined dataframes
    global user_item_interactions, df_books_final
    
    # Step 1: Encode the customer_id to retrieve the index of customer
    encoder = LabelEncoder()
    user_item_interactions['user_id'] = encoder.fit_transform(user_item_interactions['user_id'])
    customer_idx = encoder.transform([customer_id])[0]
    
    # Find list of items already interacted by the query customer
    interacted_items = user_item_interactions[user_item_interactions['user_id'] == customer_id]['book_id'].tolist()

    # Step 2: Retrieve all similarity scores between queried user and all other customers
    similarity_scores = user_similarity_matrix[customer_idx]
    # return similarity_scores.toarray()[0]

    # Step 3-4: Filter away users with perfect similarity scores (>= 0.95) & similarity scores < sim_threshold (0.6)
    similarity_scores[similarity_scores >= 0.95] = 0
    similarity_scores[similarity_scores < sim_threshold] = 0
    similar_customers_idx = similarity_scores.nonzero()[1]

    # Step 5: Find the most interacted items that were interacted by the most similar neighbours
    similar_interacted_items = user_item_interactions[user_item_interactions['user_id'].isin(similar_customers_idx)]

    # # Accounting for frequency interacted by similar customers
    # most_interacted_items = similar_interacted_items.groupby('book_id')['frequency'].sum().reset_index(name='count')

    # # Remove items already interacted by the query customer
    # new_interacted_items = most_interacted_items[~most_interacted_items['book_id'].isin(interacted_items)]

    # # Step 6: Retrieve Top K most interacted items
    # top_k_items = new_interacted_items.nlargest(k, 'count')['book_id'].tolist()
    # recommendations = df_books_final[df_books_final['book_id'].isin(top_k_items)]



    # Calculate a score based on average rating - can add in age of reviews here or maybe n_votes?
    weighted_items = similar_interacted_items.groupby('book_id').agg(
        avg_rating=('rating', 'mean')
    ).reset_index()

    # Define weights for frequency and average rating
    # freq_weight = 0.5
    # rating_weight = 0.5
    
    # Calculate the combined score
    # weighted_items['combined_score'] = (
    #     freq_weight * weighted_items['frequency_sum'] + rating_weight * weighted_items['avg_rating']
    # )

    # Remove items already interacted with by the query customer
    new_interacted_items = weighted_items[~weighted_items['book_id'].isin(interacted_items)]

    # Step 6: Retrieve Top K items based on the combined score
    top_k_items = new_interacted_items.nlargest(k, 'avg_rating')['book_id'].tolist()
    recommendations = df_books_final[df_books_final['book_id'].isin(top_k_items)]

    return recommendations

    # return similarity_scores.toarray()[0]
    # return interacted_items

### Recommend Book

In [104]:
len(df_train_interactions[df_train_interactions['user_id'] == 2])

52

In [58]:
# test similaritiy scores
items_to_recommend = get_collaborative_filtering_recommendations(user_similarity_matrix, 2, k, sim_threshold)
items_to_recommend[np.nonzero(items_to_recommend)[0]]

NameError: name 'user_similarity_matrix' is not defined

In [123]:
items_to_recommend = get_collaborative_filtering_recommendations(user_similarity_matrix, 2, k, sim_threshold)
print('Number of recommendations:', len(items_to_recommend))
items_to_recommend

Number of recommendations: 0


C:\Users\rayna\AppData\Local\Temp\ipykernel_40120\2936419507.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_item_interactions['user_id'] = encoder.fit_transform(user_item_interactions['user_id'])
C:\Users\rayna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\sparse\_compressed.py:290: SparseEfficiencyWarning: Comparing a sparse matrix with a scalar greater than zero using < is inefficient, try using >= instead.
  warn(bad_scalar_msg, SparseEfficiencyWarning)
C:\Users\rayna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing 

,book_id,average_rating,ratings_count,text_reviews_count,top_popular_shelves,author_ids,title_tfidf,description_word2vec,format_Audio,format_Digital,...,lang_tr,lang_tur,lang_ukr,lang_vi,lang_vie,lang_yid,lang_zh,length_long,length_medium,length_short


In [51]:
def read_titles(df):
    """
    Read the mapping of book id -> book title

    Returns a dictionary
    {book id -> book title}
    """
    data = df.set_index("book_id")["title"].to_dict()
    return data

In [285]:
# book_titles = read_titles(df_books_final)

In [84]:
def recommend(user_id, ratings, book_titles, n_neighbors=10, n_recomm=5):
    """

    Recommend N books for a given user based on ratings data.

    1. get the ratings of the user
    2. get the books that the user has not rated
    3. compute the similarities between the user and the other users
    4. generate book ratings predictions for the user based on the similarities with other users
    5. find the N books with the highest predicted ratings

    parameters:
    - user_id: int, user to generate recommendations for
    - ratings: pd.DataFrame, user-book ratings
    - book_titles: dict, mapping of (book id -> book title)
    - n_neighbors: int: the number of neighbors to use to generate rating predictions
    - n_recomm: int, number of books to recommend

    returns:
    - pd.DataFrame with [book_id, rating, book title]

    """

    # get the ratings of the user
    ratings_user = ratings.loc[user_id, :]

    # all the items a user has not rated, that can be recommended
    all_items = ratings.loc[user_id,:]
    unrated_items = all_items.loc[all_items.isnull()]

    # convert the index with item ids into Series values
    unrated_items = unrated_items.index.to_series(name='item_ids').reset_index(drop=True)
    print(unrated_items)
    print('User {} has {} unrated items.'.format(user_id, len(unrated_items)))

    # compute user similarities
    similarities = compute_similarities(user_id, ratings)
    # print(similarities.columns)
    sorted = similarities.sort_values(by="similarity", ascending=False)
    # print(sorted)
    # return similarities

    # generate predictions for unseen items based on the user similarity data
    predictions = unrated_items.apply(lambda d: predict_rating(d, ratings, similarities, N=n_neighbors))
    return(predictions)

    # sort items by highest predicted rating and number of similar users
    predicitions_df = predicitions_df.sort_values(by=['predictions', 'number_of_similar_users'], ascending=False)

    # recommend top N items
    recommends = predicitions_df.head(n_recomm)

    # reformat the result
    recommends = recommends.to_frame(name='predicted_rating')
    recommends = recommends.rename_axis('book_id')
    recommends = recommends.reset_index()

    # recommends['name'] = recommends.book_id.apply(lambda d: book_titles[d])

    # return recommends

In [85]:
recommends = recommend(1, affinity_matrix, book_titles={}, n_neighbors=N_NEIGHBORS, n_recomm=N_RECOMMENDATIONS)
recommends

AttributeError: 'csr_matrix' object has no attribute 'loc'

### Recommend for users in test set

In [291]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_scores =  tfidf.fit_transform(df_books_final['description'])
tfidf_scores.shape

(59828, 149225)

In [ ]:
# cosine similarity matrix directly for the sparse TF-IDF matrix
tfidf_cos_sim = cosine_similarity(tfidf_scores, dense_output=False)

In [275]:
print(df_books_final.index[df_books_final['book_id'] == 46677])

Index([59785], dtype='int64')


In [288]:
book_index = df_books_final.index[df_books_final['book_id'] == 343002][0]
similarity_scores = tfidf_cos_sim[book_index].toarray().flatten()
index = df_books_final.index[df_books_final['book_id'] == 46677][0]
print(similarity_scores[index])
index = df_books_final.index[df_books_final['book_id'] == 867248][0]
print(similarity_scores[index])
index = df_books_final.index[df_books_final['book_id'] == 6669717][0]
print(similarity_scores[index])
index = df_books_final.index[df_books_final['book_id'] == 22013040][0]
print(similarity_scores[index])
index = df_books_final.index[df_books_final['book_id'] == 402114][0]
print(similarity_scores[index])
print("break")
book_index = df_books_final.index[df_books_final['book_id'] == 1852][0]
similarity_scores = tfidf_cos_sim[book_index].toarray().flatten()
index = df_books_final.index[df_books_final['book_id'] == 46677][0]
print(similarity_scores[index])
index = df_books_final.index[df_books_final['book_id'] == 867248][0]
print(similarity_scores[index])
index = df_books_final.index[df_books_final['book_id'] == 6669717][0]
print(similarity_scores[index])
index = df_books_final.index[df_books_final['book_id'] == 22013040][0]
print(similarity_scores[index])
index = df_books_final.index[df_books_final['book_id'] == 402114][0]
print(similarity_scores[index])


0.02697621518236352
0.0
0.03242136805362899
0.002300617889906126
0.011088978479916148
break
0.03906784168008337
0.015002949668787525
0.037975677305821874
0.007384900175627537
0.011830229507892809


In [283]:
unique_users = df_test_interactions.user_id.unique()
recommendations = []
total = 0
count = 0
for user in unique_users:
    count += 1
    recommends = recommend(user, ratings, n_neighbors=N_NEIGHBORS, n_recomm=N_RECOMMENDATIONS)
    # print(recommends)
    test_books = df_test_interactions[df_test_interactions['user_id'] == user]['book_id']
    # print(test_books)
    
    print(test_books.isin(recommends).sum())
    total += test_books.isin(recommends).sum()
    if count ==50:
        print(f"TOTAL: {total}")
        break
    # break

User 0 has 14638 unrated items.


/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  

0
User 1 has 14635 unrated items.


/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  

0
User 2 has 14638 unrated items.


...

0
User 48 has 14635 unrated items.


/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  

0
User 49 has 14633 unrated items.


/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/homebrew/Caskroom/miniforge/base/envs/env_bt4222Project/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  

0
TOTAL: 0
